In [13]:
from __future__ import print_function, division

from tensorflow import keras

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import cv2
import pandas as pd

In [2]:
def create_dataset():
    imgs = []
    for i in range(11024):
        img = cv2.imread('train_1/img_{}.jpg'.format(i))
        if img is not None:
            imgs.append(cv2.resize(img, (256,256), interpolation=cv2.INTER_AREA))
    
    return np.array(imgs)

In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        X_train = create_dataset()


        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)


        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan_images/%d.png" % epoch)
        plt.close()


In [4]:
gan = GAN()
gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               100663808 
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 100,795,393
Trainable params: 100,795,393
Non-trainable params: 0
______________________________________________________________

KeyboardInterrupt: 

In [11]:
# apply a 3x3 transposed convolution
# with stride 1x1 and 3 output filters on a 12x12 image:
model = keras.models.Sequential()
model.add(keras.layers.Conv2DTranspose(3, 2, 2,
              padding='valid',
              input_shape=(64, 64, 3)))
# Note that you will have to change
# the output_shape depending on the backend used.

# we can predict with the model and print the shape of the array.
dummy_input = np.ones((32, 64, 64, 3))
# For TensorFlow dummy_input = np.ones((32, 12, 12, 3))
preds = model.predict(dummy_input)
print(preds.shape)

(32, 128, 128, 3)


In [14]:
image_data = pd.read_csv('data/train_info.csv')

In [16]:
image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79433 entries, 0 to 79432
Data columns (total 6 columns):
filename    79433 non-null object
artist      79433 non-null object
title       79420 non-null object
style       78668 non-null object
genre       78540 non-null object
date        59178 non-null object
dtypes: object(6)
memory usage: 3.6+ MB


In [35]:
styles = image_data['style'].unique()

In [23]:
genres = image_data['genre'].unique()

In [36]:
styles = pd.DataFrame(styles, columns=['style'])

In [39]:
styles['count'] = [np.sum(image_data['style']==s) for s in styles['style'].values]

In [42]:
genres = pd.DataFrame(genres, columns=['genre'])

In [43]:
genres['count'] = [np.sum(image_data['genre']==g) for g in genres['genre'].values]

In [44]:
genres

,genre,count
0,abstract,7201
1,mythological painting,1493
2,bird-and-flower painting,100
3,landscape,11548
4,religious painting,5703
5,marina,1385
6,genre painting,10984
7,still life,2464
8,animal painting,1233
9,self-portrait,1199
